# Mitochondria

Analysis for mitochondria proposals

In [1]:
from pathlib import Path
import sys

from plotly.graph_objs import Figure

debug_local = True#False
local = (Path("..") / "mitochondria").resolve()
if debug_local and local.exists():
    sys.path.insert(0, Path("..").as_posix())
    print("extending pathes with local indexpaper")
    print(sys.path)
    %load_ext autoreload
    %autoreload 2

extending pathes with local indexpaper
['..', '/home/antonkulaga/sources/mitochondria/notebooks', '/home/antonkulaga/sources/mitochondria', '/home/antonkulaga/micromamba/envs/mitochondria/lib/python310.zip', '/home/antonkulaga/micromamba/envs/mitochondria/lib/python3.10', '/home/antonkulaga/micromamba/envs/mitochondria/lib/python3.10/lib-dynload', '', '/home/antonkulaga/.local/lib/python3.10/site-packages', '/home/antonkulaga/micromamba/envs/mitochondria/lib/python3.10/site-packages']


In [2]:
from pycomfort.files import *
base = Path("..")
data = base / "data"
mammals = data / "mammals"
mammals

PosixPath('../data/mammals')

In [3]:
tprint(mammals)

mammals
	Macaca_mulatta.Mmul_10.dna_rm.primary_assembly.MT.fa
	Pan_troglodytes.Pan_tro_3.0.dna.chromosome.MT.fa
	Gorilla_gorilla.gorGor4.dna.chromosome.MT.fa
	Homo_sapiens.GRCh38.dna.chromosome.MT.fa


# MSA

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os

In [5]:
from Bio.Align.Applications import ClustalwCommandline
from Bio import AlignIO
from pathlib import Path
import tempfile
import plotly.graph_objects as go

def perform_msa(folder_path: Path, output_file: Path):
    # Check if the given path is a directory
    if not folder_path.is_dir():
        raise ValueError(f"{folder_path} is not a directory")

    # Get all .fa and .fasta files in the directory
    input_files = list(folder_path.glob('*.fa')) + list(folder_path.glob('*.fasta'))

    # Create a temporary file to concatenate the sequences
    with tempfile.NamedTemporaryFile(mode='w+', delete=False) as temp_file:
        for file_path in input_files:
            # Determine new sequence identifier based on file name
            new_seq_id = "MT_" + file_path.stem.split(".")[0]

            # Read and modify the content of the file before appending to the temporary file
            with open(file_path, 'r') as f:
                content = f.read()
                content = content.replace(">MT ", f">{new_seq_id} ")
                temp_file.write(content)
                temp_file.write("\n")  # Just to ensure sequences are separated

        temp_input_file = Path(temp_file.name)

    # Setup the ClustalW command
    clustalw_cline = ClustalwCommandline("clustalw2", infile=temp_input_file, outfile=output_file)

    # Run the command
    stdout, stderr = clustalw_cline()

    # Clean up the temporary file
    temp_input_file.unlink()

    return output_file

In [6]:
folder_path = mammals
# Output alignment file path
output_file = Path("mammals_alignment.aln")

# Perform the MSA
alignment_file = perform_msa(folder_path, output_file)

In [7]:
data_path = os.path.join(matplotlib.get_configdir(), 'mpl-data')
print(data_path)

/home/antonkulaga/.config/matplotlib/mpl-data


In [8]:
from Bio import AlignIO
from collections import defaultdict
from itertools import combinations

def generate_alignment_statistics(alignment_file: str) -> dict:
    # Load the alignment
    alignment = AlignIO.read(alignment_file, "clustal")

    # Prepare a dictionary to store results
    stats = defaultdict(lambda: {'matches': 0, 'differences': 0})

    # Iterate through each pair of sequences in the alignment
    for record1, record2 in combinations(alignment, 2):
        id1, id2 = record1.id, record2.id
        for base1, base2 in zip(record1.seq, record2.seq):
            # Ignore comparison for positions with gaps in either sequence
            if base1 == "-" or base2 == "-":
                continue
            if base1 == base2:
                stats[(id1, id2)]['matches'] += 1
            else:
                stats[(id1, id2)]['differences'] += 1

    return stats

In [10]:
import polars as pl
from Bio import pairwise2
from Bio.SubsMat import MatrixInfo
from typing import Tuple, List

def calculate_pairwise_statistics(seq1: str, seq2: str) -> Tuple[int, int, int]:
    alignments = pairwise2.align.globalds(seq1, seq2, MatrixInfo.blosum62, -10, -0.5)
    align1, align2, score, begin, end = alignments[0]

    substitutions, insertions, deletions = 0, 0, 0
    for a, b in zip(align1, align2):
        if a == '-' and b != '-':
            insertions += 1
        elif a != '-' and b == '-':
            deletions += 1
        elif a != b:
            substitutions += 1

    return substitutions, insertions, deletions

def pairwise_statistics_for_genes(df: pl.DataFrame) -> pl.DataFrame:
    sequence_columns = [col for col in df.columns if "_sequence" in col]
    results = []

    for row in df.rows():
        gene_name = row[0]  # Assuming gene_name is the first column
        sequences = row[1:]
        for i in range(len(sequence_columns)):
            for j in range(i+1, len(sequence_columns)):
                seq_i = sequences[i]
                seq_j = sequences[j]

                subs, ins, dels = calculate_pairwise_statistics(seq_i, seq_j)
                results.append({
                    "gene_name": gene_name,
                    "pair": f"{sequence_columns[i].split('_')[0]}_vs_{sequence_columns[j].split('_')[0]}",
                    "substitutions": subs,
                    "insertions": ins,
                    "deletions": dels
                })

    return pl.DataFrame(results)